In [67]:
import gurobipy as gp
from gurobipy import GRB

In [97]:
#Convertir a dictionary
#dist = dict(enumerate(matriz_custos_1.flatten(), 1))
matriz_custos_1.keys()

AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [69]:
# Variables: is city 'i' adjacent to city 'j' on the tour?
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

NameError: name 'dist' is not defined

In [66]:
#
num_vertex = 250#Gian
# tested with Python 3.7 & Gurobi 9.0.0
m = gp.Model()

# Variables: is city 'i' adjacent to city 'j' on the tour?
vars = m.addVars(num_vertex, num_vertex, vtype=GRB.BINARY, name='x')#Gian
# TODO adicionar o segundo tuor
# TODO adicionar as variáveis D_e

# Symmetric direction: Copy the object
for i, j in vars.keys():
    vars[j, i] = vars[i, j]  # edge in opposite direction

#---------------------------------------------------------------------------------
#RESTRICTION 1: no input can be from a vertex for the same
m.addConstr(gp.quicksum([vars[i,j] for i in range(num_vertex)])==0)
#RESTRICTION 2: 


#---------------------------------------------------------------------------------

# Constraints: two edges incident to each city
cons = m.addConstrs(vars.sum(c, '*') == 2 for c in capitals)

# TODO adicionar as restrições de D_e

# Callback - use lazy constraints to eliminate sub-tours

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(capitals):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = capitals[:]
    cycle = capitals[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

# Retrieve solution

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(capitals)

NameError: name 'dist' is not defined

In [89]:
import numpy as np
#Ler arquivo
coordenadas = open("./mo824_atividade2_coords", "r")
dimensiones_coordenadas=(250,4)
matriz_coordenadas = np.zeros(dimensiones_coordenadas)
#Loop de 0 a 250
for i in range(dimensiones_coordenadas[0]):
    matriz_coordenadas[i] = coordenadas.readline().split()
#matriz_coordenadas    

In [90]:
#Funçao custo
def f_custo(x_i, y_i, x_j, y_j):
    return np.sqrt(np.power(x_i-x_j,2)+np.power(y_i-y_j,2))
#f_custo(0,3,4,0)

In [91]:
dimensiones_custos = (250,250)
matriz_custos_1 = np.zeros(dimensiones_custos)
matriz_custos_2 = np.zeros(dimensiones_custos)
for i in range(dimensiones_custos[0]):
    for j in range(dimensiones_custos[1]):
        if i != j :
            matriz_custos_1[i,j] = f_custo(matriz_coordenadas[i,0], matriz_coordenadas[i,1], matriz_coordenadas[j,0], matriz_coordenadas[j,1])
            matriz_custos_2[i,j] = f_custo(matriz_coordenadas[i,2], matriz_coordenadas[i,3], matriz_coordenadas[j,2], matriz_coordenadas[j,3])
#matriz_custos_1